In [1]:
import pandas as pd
import numpy as np
import requests
import csv
import folium
import ast

In [3]:
# Load the data
df = pd.read_excel('MergedData.xlsx')

# List of columns that contain hospital names
columns = ['Providers_08_09', 'Providers_09_10', 'Providers_10_11', 'Providers_11_12', 
           'Providers_12_13', 'Providers_13_14', 'Providers_14_15', 'Providers_15_16', 
           'Providers_16_17', 'Providers_17_18', 'Providers_18_19', 'Providers_19_20', 
           'Providers_20_21', 'Providers_21_22']

# Collect all unique hospital names
hospitalsList = pd.unique(df[columns].values.ravel('K'))

# Print the list of unique hospital names
print(hospitalsList)


[nan 'South Gloucestershire PCT' 'Havering PCT' ...
 'University Hospitals Sussex NHS Foundation Trust'
 'Highgate Hospital Gender Service' 'Spamedica Gateshead']


In [4]:
# Remove nan values from the list
hospitalsList = [hospital for hospital in hospitalsList if not pd.isnull(hospital)]

In [5]:
print(len(hospitalsList))

2736


In [6]:
hospitalsList

['South Gloucestershire PCT',
 'Havering PCT',
 'Kingston PCT',
 'Hillingdon PCT',
 'Enfield PCT',
 'Tower Hamlets PCT',
 'Newham PCT',
 'Haringey Teaching PCT',
 'Herefordshire PCT',
 'Milton Keynes PCT',
 'Plymouth Teaching PCT',
 'Portsmouth City Teaching PCT',
 'Bath And North East Somerset PCT',
 'Rotherham PCT',
 'Warrington PCT',
 'Barnsley PCT',
 'Swindon PCT',
 'Brent Teaching PCT',
 'Camden PCT',
 'South Tyneside PCT',
 'Sunderland Teaching PCT',
 'Southampton City PCT',
 'Medway PCT',
 'Kensington And Chelsea PCT',
 'Wandsworth PCT',
 'South Birmingham PCT',
 'Shropshire County PCT',
 'Walsall Teaching PCT',
 'Richmond And Twickenham PCT',
 'Sutton And Merton PCT',
 'Telford And Wrekin PCT',
 'Wolverhampton City PCT',
 'Heart Of Birmingham Teaching PCT',
 'Doncaster PCT',
 'Derbyshire County PCT',
 'Nottinghamshire County Teaching PCT',
 'Cumbria Teaching PCT',
 'Central Lancashire PCT',
 'Liverpool PCT',
 'Western Cheshire PCT',
 'North Yorkshire And York PCT',
 'East Ridin

In [7]:
# Create a new list with only those names that contain 'Hospital', case-insensitive
hospitals_with_word = [name for name in hospitalsList if isinstance(name, str) and 'hospital' in name.lower()]

# Print the new list
print(hospitals_with_word)

['Nuffield Health, Plymouth Hospital', 'Spire South Bank Hospital', 'Spire Bristol Hospital', 'Spire Southampton Hospital', 'Spire Portsmouth Hospital', 'Spire Gatwick Park Hospital', 'Spire Tunbridge Wells Hospital', 'Spire Alexandra Hospital', 'Spire Wellesley Hospital', 'Spire Roding Hospital', 'Spire Bushey Hospital', 'Spire Harpenden Hospital', 'Spire Cambridge Lea Hospital', 'Spire Norwich Hospital', 'Spire Hartswood Hospital', 'Spire Parkway Hospital', 'Spire Little Aston Hospital', 'Spire Leicester Hospital', 'Spire Cheshire Hospital', 'Spire Murrayfield Hospital', 'Spire Manchester Hospital', 'Spire Leeds Hospital', 'Spire Washington Hospital', 'Spire Liverpool Hospital', 'Spire Regency Hospital', 'BMI South Cheshire Private Hospital (Spire)', 'Spire Thames Valley Hospital', 'Spire Dunedin Hospital', 'Spire Clare Park Hospital', 'Spire St Saviours Hospital', 'Spire Fylde Coast Hospital', 'Spire Elland Hospital', 'Spire Methley Park Hospital', 'Spire Hull And East Riding Hospit

In [8]:
print(len(hospitals_with_word))

1391


In [9]:
hospitals = hospitalsList[:100]

In [10]:
def search_place(api_key, query):
    url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={query}&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry,place_id&key={api_key}&language=en"
    response = requests.get(url)
    data = response.json()
    if 'candidates' in data and len(data['candidates']) > 0:
        place = data['candidates'][0]
        return place
    else:
        return None

api_key = ''

# Open (or create) a CSV file for writing
with open('hospital_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['name', 'address', 'coordinates', 'place_id']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    not_found_counter = 0  # Counter for places not found
    
    for hospital in hospitals:
        query = hospital
        try:
            place = search_place(api_key, query)
            if place:
                print(f"Place: {place['name']}")
                print(f"Address: {place['formatted_address']}")
                print(f"Coordinates: {place['geometry']['location']}")
                print(f"Place ID: {place['place_id']}")

                # Write the place data to the CSV file
                writer.writerow({'name': place['name'], 'address': place['formatted_address'], 'coordinates': place['geometry']['location'], 'place_id': place['place_id']})
            else:
                not_found_counter += 1
                print(f"Could not find place for {query}. Total not found: {not_found_counter}")
        except UnicodeEncodeError:
            print(f"UnicodeEncodeError occurred for {query}. Skipping this entry.")


Place: Bristol Community Health PCT
Address: Southmead Health Centre, Southmead, Bristol BS10 6DF, United Kingdom
Coordinates: {'lat': 51.5045623, 'lng': -2.601432}
Place ID: ChIJ1ft7yx-ScUgRVmy2NXHoFVU
Place: Havering Clinical Commissioning Group
Address: 6th Floor, North House, St Edwards Way, Romford RM1 3AE, United Kingdom
Coordinates: {'lat': 51.57991939999999, 'lng': 0.1795954}
Place ID: ChIJK6qXlrWk2EcRYbq7u60ODTg
Place: Get Active Kingston PCT
Address: Hollyfield House, 22 Hollyfield Rd, Surbiton KT5 9AL, United Kingdom
Coordinates: {'lat': 51.3924216, 'lng': -0.2974033}
Place ID: ChIJ8yGSgjIKdkgRQKBplSP7Gq0
Place: Hillingdon Primary Care Trust
Address: 109 High St, West Drayton UB7 7HJ, United Kingdom
Coordinates: {'lat': 51.5134986, 'lng': -0.4732120999999999}
Place ID: ChIJUazp3P9xdkgROTaEYlh92os
Place: Enfield Unity Pcn
Address: 28 Tenniswood Rd, Enfield EN1 3LL, United Kingdom
Coordinates: {'lat': 51.6625776, 'lng': -0.0769231}
Place ID: ChIJVVVhVEcfdkgR_cn3GI3rTP4
Place: 

Place: East Riding Of Yorkshire Primary Care Trust
Address: Grange Park Ln, Willerby, Hull HU10 6DT, United Kingdom
Coordinates: {'lat': 53.7631159, 'lng': -0.4535729}
Place ID: ChIJQT88VUnAeEgRX6JvpWflpWI
Place: East & North Hertfordshire NHS Trust
Address: Lister Hospital East Hall Block, Lister Hospital, Coreys Mill Lane, Coreys Mill Ln, Stevenage SG1 4AB, United Kingdom
Coordinates: {'lat': 51.92422029999999, 'lng': -0.2118406}
Place ID: ChIJVVUVBe0xdkgRAzO4b2iY1pU
Place: West Hertfordshire GP Specialist Training Programme
Address: Medical Education Centre, General Hospital, Vicarage Rd, Watford WD18 0HB, United Kingdom
Coordinates: {'lat': 51.6490376, 'lng': -0.4044799}
Place ID: ChIJPS7gpOdqdkgRE2TVozCUgEo
Place: Surrey Primary Care Trust
Address: 757 Guildford Rd, Fetcham, Leatherhead KT22 9DF, United Kingdom
Coordinates: {'lat': 51.2931219, 'lng': -0.3350435}
Place ID: ChIJgcau3OTgdUgRK1MXuMYVX-Q
Place: Sussex Partnership Foundation Trust
Address: Trust Headquarters, Arundel Rd

Place: Spire Gatwick Park Hospital
Address: Povey Cross Rd, Hookwood, Horley RH6 0BB, United Kingdom
Coordinates: {'lat': 51.167841, 'lng': -0.18338}
Place ID: ChIJe9QhjxzwdUgRJE_Vm1g27Cw
Place: Spire Tunbridge Wells Hospital
Address: Fordcombe Rd, Fordcombe, Tunbridge Wells TN3 0RD, United Kingdom
Coordinates: {'lat': 51.134127, 'lng': 0.185419}
Place ID: ChIJkY7mMitb30cRNhEnwIrk4rY
Place: Spire Alexandra Hospital
Address: Impton Ln, Walderslade, Chatham ME5 9PG, United Kingdom
Coordinates: {'lat': 51.3279434, 'lng': 0.5267913}
Place ID: ChIJ6Q10hNwy30cRZ7w6W_F4jyc
Place: Spire Wellesley Hospital
Address: Eastern Ave, Southend-on-Sea SS2 4XH, United Kingdom
Coordinates: {'lat': 51.5546679, 'lng': 0.7272029}
Place ID: ChIJ8RJA-FfY2EcRKHPyzkp4thw
Place: Spire London East Hospital (Formerly Spire Roding Hospital)
Address: Roding Ln S, Woodford, Ilford, Woodford Green IG4 5PZ, United Kingdom
Coordinates: {'lat': 51.5869904, 'lng': 0.0433908}
Place ID: ChIJxUPrRian2EcR3AR90g4TE4A
Place: Sp

In [ ]:
not_found_counter

Could not find total 477 instances 

In [2]:
# Load the CSV file
df = pd.read_csv('hospital_data.csv')

# Convert 'coordinates' from string to dictionary
df['coordinates'] = df['coordinates'].apply(ast.literal_eval)

# Extract 'lat' and 'lng' into separate columns
df['latitude'] = df['coordinates'].apply(lambda x: x['lat'])
df['longitude'] = df['coordinates'].apply(lambda x: x['lng'])

# Select the first 50 rows

# Create a map centered around the first coordinate
m = folium.Map(location=[df['latitude'].iloc[0], df['longitude'].iloc[0]], zoom_start=13)

# Add points to the map
for index, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(m)

# Display the map
m

In [13]:
# Create a new DataFrame with addresses not in the United Kingdom
df_not_uk = df[~df['address'].str.contains('United Kingdom')]

# Select the 'name' and 'address' columns
df_not_uk = df_not_uk[['name', 'address']]

# Convert the DataFrame to a list of dictionaries
names_addresses_not_in_uk = df_not_uk.to_dict('records')

In [14]:
names_addresses_not_in_uk

[{'name': 'Kensington and Chelsea',
  'address': 'Kensington and Chelsea, London SW7 4DG, UK'},
 {'name': 'Berkshire East Mountain Resort',
  'address': '66 Thunder Mountain Rd, Charlemont, MA 01339, United States'},
 {'name': 'St. Marien-Hospital Berlin',
  'address': 'Gallwitzallee 123 – 143, 12249 Berlin, Germany'},
 {'name': 'St. Catherine Hospital',
  'address': 'Seckbacher Landstraße 65, 60389 Frankfurt am Main, Germany'},
 {'name': 'Milton District Hospital',
  'address': '725 Bronte St. S, Milton, ON L9T 9K1, Canada'},
 {'name': 'The George Eliot Hospital N H S Trust',
  'address': 'The George Eliot Hospital N H S Trust, College St, Nuneaton CV10 7DJ, UK'},
 {'name': 'BMI The Kings Oak Hospital',
  'address': 'BMI The Kings Oak Hospital, Enfield EN2 8SD, UK'},
 {'name': 'Essex', 'address': 'Essex, UK'},
 {'name': 'Berkshire East Mountain Resort',
  'address': '66 Thunder Mountain Rd, Charlemont, MA 01339, United States'},
 {'name': 'Lazarus Hospiz',
  'address': 'Bernauer Str. 

In [12]:
def get_place_details(api_key, place_id):
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,rating,formatted_address,permanently_closed&key={api_key}"
    response = requests.get(url)
    data = response.json()
    if 'result' in data:
        place = data['result']
        return place
    else:
        return None

api_key = ''

# Open the CSV file for reading
with open('hospital_data.csv', 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in reader:
        place_id = row['place_id']
        place = get_place_details(api_key, place_id)
        if place:
            print(f"Name: {place['name']}")
            print(f"Address: {place['formatted_address']}")
            if 'rating' in place:
                print(f"Rating: {place['rating']}")
            else:
                print("No rating available for this place.")
            if 'permanently_closed' in place:
                print(f"Permanently Closed: {place['permanently_closed']}")
            else:
                print("This place is not permanently closed.")
        else:
            print(f"Could not find place for place_id {place_id}.")



Name: Bristol Community Health PCT
Address: Southmead Health Centre, Southmead, Bristol BS10 6DF, UK
Rating: 2
This place is not permanently closed.
Name: Havering Clinical Commissioning Group
Address: 6th Floor, North House, St Edwards Way, Romford RM1 3AE, UK
Rating: 3.5
This place is not permanently closed.
Name: Get Active Kingston PCT
Address: Hollyfield House, 22 Hollyfield Rd, Surbiton KT5 9AL, UK
No rating available for this place.
This place is not permanently closed.
Name: Hillingdon Primary Care Trust
Address: 109 High St, West Drayton UB7 7HJ, UK
No rating available for this place.
This place is not permanently closed.
Name: Enfield Unity Pcn
Address: 28 Tenniswood Rd, Enfield EN1 3LL, UK
No rating available for this place.
This place is not permanently closed.
Name: Tower Hamlets Primary Care Trust
Address: 275 Bancroft Rd, Bethnal Green, London E1 4DG, UK
No rating available for this place.
Permanently Closed: True
Name: Newham Transitional Practice
Address: 30 Church Rd,

Name: PCT Coaching
Address: Unit 5B, Sylvan Centre, off Sheaf Passage, Webberley Ln, Longton, Stoke-on-Trent ST3 1RJ, UK
Rating: 5
This place is not permanently closed.
Name: South Staffordshire PCT
Address: Cross St, Burton upon Trent, Burton-on-Trent DE14 1EG, UK
Rating: 5
This place is not permanently closed.
Name: Worcestershire P C T
Address: 30 Crossgate Rd, Park Farm Industrial Estate, Redditch B98 7SN, UK
No rating available for this place.
This place is not permanently closed.
Name: South Warwickshire Primary Care Trust
Address: Heathcote La, Heathcote, Warwick CV34 6SR, UK
No rating available for this place.
This place is not permanently closed.
Name: Cambridgeshire and Peterborough ICS
Address: Gemini House Cambridgeshire, Business Park, Ely CB7 4EA, UK
Rating: 5
This place is not permanently closed.
Name: Norfolk Primary Care Trust
Address: Elliott House, 130 Ber St, Norwich NR1 3FR, UK
No rating available for this place.
Permanently Closed: True
Name: Hertfordshire & West 